# Atomic relaxation data

## Interface overview

[Transition data](atomic-relaxation-data/1-transition-data.ipynb)

[Electron subshell configuration data](atomic-relaxation-data/2-subshell-configuration-data.ipynb)
